<a href="https://colab.research.google.com/github/emgoss/PLP6621C/blob/main/Module5_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 5 Assignment: Recombination

**Purpose:**
The purpose of this assignment is to learn how to calculate and interpret the index of association for microsatellite and SNP data sets. This statistic has been used for both bacterial and fungal populations to test for significant linkage disequilibrium. This assignment contributes to overall course objectives of discussing how analysis of population genetic data can be used to learn about microbial populations, evaluating the assumptions, limitations, and appropriate use of population genetic analysis in the plant pathology literature, and conducting conduct population genetic analyses using existing data sets.


We will use R packages for this assignment. The index of association for SSRs analysis is taken verbatum from this tutorial, so please also refer to the source material:
https://grunwaldlab.github.io/Population_Genetics_in_R/index.html

For the SNP data, I have also used this:
https://grunwaldlab.github.io/poppr/articles/mlg.html
and Step5_Data_processing.Rmd from here:
https://github.com/grunwaldlab/rubi-gbs


## Calculating the Index of Association with Microsatellite Data

**1. Install the first R package we need, [poppr](https://grunwaldlab.github.io/poppr/reference/poppr-package.html).**

In [ ]:
install.packages("poppr")
library("poppr")

**2. Load the data**

The data used in this example, called monpop, are from Everhart & Scherm, 2015. This is a microsatellite data set  of 13 loci for 694 individuals of the haploid fungal pathogen *Monilinia fructicola* that infects peach flowers and fruits in commercial orchards. The data came from four trees within a single orchard (trees 26, 45, and 7). Each tree was sampled in 2009, 2010, and/or 2011. Additionally, each sample was noted as to whether it came from a blossom or a fruit. This example data set is included with the poppr package.

Poppr allows setting the hierachy of your data so that you can easily analyze different population or grouping levels. These are called "strata".

The steps for working with stratified data include:

1. Import data set with samples labeled according to strata
2. Define the stratifications for the data
3. Setting the stratification(s) that you want to have analyzed

The easiest way to work with stratified data is to label each sample using an underscore “_” to separate each level. This was already done for the monpop data, where each sample was coded hierarchically by tree, year, and symptom in the following format: “Tree_Year_Symptom”.

The data we will use is distributed with the poppr package so we don't need to import it, just load it.


In [ ]:
data(monpop)
monpop #view a summary of the data

The **genclone object** is a type of container that holds the data. In this case, it structured for small genotype data sets that will be clone corrected.

**Genotype information** shows us that the data contains 264 multilocus genotypes among 694 haploid individuals with 13 loci.

**Population information** has two items, the stratifications and the populations defined. You can think of stratifications as the index names for each of the hierarchical levels within your data (so for our data it should be Tree, Year, and Symptom). By default, however, no stratifications are defined and so this is “Pop”, which is the entire dataset of 694 individuals. Because we labeled each sample according to stratification, populations defined shows us our data has 12 groups defined: 7_09_BB, 26_09_BB, 26_09_FR, 7_09_FR, 26_10_BB, 45_10_BB, 79_10_BB, 79_10_FR, 26_10_FR, 45_10_FR, 26_11_BB, and 26_11_FR.



**3. Assign stratifications**

We imported the data that has three stratifications “Tree_Year_Symptom”. In order to analyze our data according to any combination of those three stratifications, we need to tell poppr that the 12 groups should be split by tree, year, and/or symptom. Thus, the first step is to split our data according to strata so that we can access each of the three hierarchical levels in the data. The splitStrata command is used to index the three stratifications:

In [ ]:
splitStrata(monpop) <- ~Tree/Year/Symptom
monpop # view the data summary again

What changed?

After splitting the data, there are 3 strata: “Tree Year Symptom”.

**4. Setting different strata (populations)**

To analyze the data according to Symptom:

In [ ]:
setPop(monpop) <- ~Symptom
monpop # check the populations

Now there are two populations, blossom blight (BB) and fruit rot (FR).

If we wanted to define the symptoms according to tree, we would use the following:

In [ ]:
setPop(monpop) <- ~Symptom/Tree
monpop

Note that these populations combine the three different years.

**5. Clone correction**

When dealing with clonal populations, analyses are typically conducted with and without clone correction. Clone correction is a method of censoring a data set such that only one individual per MLG is represented per population (Milgroom, 1996; Grünwald et al., 2003; Grünwald & Hoheisel, 2006). This technique is commonly used with the index of association and genotypic diversity measures since clone corrected populations approximate behavior of sexual populations. Since we want to only observe unique genotypes per population, clone correction requires specification of the stratifications (population) at which clones should be censored. This section will show how to clone correct at a specific stratification and also compare the results with uncorrected data.

Looking at the above summaries of the monpop data set, we see that there are only 264 genotypes in the total sample of 694 individuals. This indicates that there are many individuals that share identical multilocus genotypes.

We then have to decide how we want to clone correct, which must be informed by the biology of the organism and sampling strategy. Take a break from the code and **read through the sampling methods** https://apsjournals.apsnet.org/doi/pdf/10.1094/PHYTO-03-14-0088-R. What is the most biologically appropriate way to clone-correct these data?


We will try clone correcting using different population definitions and compare the results. To set our populations to tree and year, we will use the “Tree/Year” strata then clone correct, assigning the clone corrected data to a new variable so that we don't write over the original data set.

In [ ]:
setPop(monpop) <- ~Tree/Year
monpop # check that populations are defined as you wanted

In [ ]:

mcc_TY <- clonecorrect(monpop, strata = ~Tree/Year, keep = 1:2) # clone correct by both tree and year and assign to mcc_TY
mcc_TY # view the new data set

Now there are only 278 individuals because only one individual of each genotype has been retained per tree in each year.

Now clone correct by year and symptom.


In [ ]:
setPop(monpop) <- ~Year/Symptom
monpop # check that populations are defined as you wanted

In [ ]:
mcc_YS <- clonecorrect(monpop, strata = ~Year/Symptom, keep=1:2) # clone correct by both year and symptom and assign to mcc_TY
mcc_YS # view the new data set

**Question 1:**
Compare results of clone correction using the above different population definitions (strata). Explain why the number of individuals differ among different population definitions.



**6. Index of association.**

Calculate the index of association (Ia) for the 9_BB population. First we must pull the population out from the monpop dataset, then calculate the Ia. (We have to put a letter in front of the population name to not get an error in R.)

The actual value of Ia will be one of 1000 calculations of Ia. The rest use 999 random permutations of the data among individuals. This will give a minimum p-value of 0.001. If we want to calculate to more decimal places, we need to add an order of magnitude more random samples.

The output includes the value of Ia and associated p-value from the permutations (p.Ia), and rbarD and associated p-value (p.rD). Remember that rbarD takes similar values across different numbers of loci whereas the magnitude of Ia depends on the number of loci genotyped.

In [ ]:
T9_BB <- popsub(monpop, "9_BB")
ia(T9_BB, sample = 999)

Now do the same for the *clone corrected* 9_BB population.

In [ ]:
T9_BBcc <- popsub(mcc_YS, "9_BB")
ia(T9_BBcc, sample = 999)

The values of Ia and rbarD are lower for the clone corrected sample, but linkage equilibrium is rejected for both.

You can also quickly obtain the values for Ia and rbarD using the command "poppr". But you can't get p-values this way. We will talk about the other values in this table in Module 7.

In [ ]:
poppr(monpop)

In [ ]:
poppr(mcc_YS)

The lowest values of Ia and rbarD were obtained for 11_BB. Note that for this populations, N = MLG meaning that each sampled isolate had a different genotype. Therefore, the values for the clone corrected 11_BB is the same as the original 11_BB. Let's see if we have significant linkage disequilibrium in this population.

In [ ]:
T11_BB <- popsub(monpop, "11_BB")
ia(T11_BB, sample = 999)

There is not significant linkage disequilibrium in the 11_BB population.

**Question 2:**
Practice calculating the index of association. Clone correct by *year*, calculate the index of association for each year including the significance test. Summarize the results in a table. What is your interpretation of these results?

**Question 3:**
What biological explanations could explain the observed results? You may use the Everhart and Scherm paper for help.

# Index of association using SNP data

For SNP data, we need to take another approach to clone correction, which is to identify multilocus lineages rather than multilocus genotypes.

We will use a data set of 94 samples of the red raspberry pathogen *Phytophthora rubi* (Tabima et al., 2018). This pathogen is diploid oomycete. Populations were obtained by sampling individual pathogen strains from roots of infected red raspberry in the states of California (CA), Oregon (OR), and Washington (WA). Currently, there is little information about the population structure of *P. rubi* in the western USA.

Note that the Phytophthora rubi files we're using are from the Population Genetics in R tutorial, not the exact analysis in the paper.

To obtain variant calls in form of VCF data, the FASTQ reads from Illumina sequencing were mapped to the reference genome of *P. rubi* (Tabima et al., 2018) using bowtie2 (Langmead & Salzberg, 2012). Variants were called using the GATK HaplotypeCaller (McKenna et al., 2010). This data was further filtered in vcfR using read depths (DP) and mapping qualities (MQ).

**7. Install packages for importing the data and graphing the results.**

In [ ]:
install.packages(c("vcfR","RColorBrewer","ggplot2"))
library(vcfR)
library(RColorBrewer)
library(ggplot2)

**8. First import the dataset into your Colab.**

In [ ]:
# Define the URL of file
data_url <- "https://grunwaldlab.github.io/Population_Genetics_in_R/prubi_gbs.vcf.gz"

# Define the destination path and filename
data_path <- "./prubi_gbs.vcf.gz"

# Download in binary mode
download.file(url = data_url, destfile = data_path, mode = "wb")

**9. Read SNP data from the compressed VCF file (we will talk about these in Module 6) using package vcfR.**

In [ ]:
rubi.vcf <- read.vcfR("prubi_gbs.vcf.gz")
rubi.vcf


The file population_data.gbs.txt is a tab-delimited text file that includes the name of the sample, country of origin, and the population from where it was sampled. The file is available for download at: population_data.gbs.txt. This link will likely open the data in a browser. Save the data onto your hard-drive as an ASCII text file with the same name.

**10. Now download and import the metadata (population assignments).**

In [ ]:
file_url <- "https://grunwaldlab.github.io/Population_Genetics_in_R/population_data.gbs.txt"
dest_path <- "./population_data.gbs.txt"
download.file(url = file_url, destfile = dest_path, method = "auto")

In [ ]:
pop.data <- read.table("population_data.gbs.txt", sep = "\t", header = TRUE)
pop.data

**11. Convert data format for analysis.**

For the analysis, we need to convert the vcfR object into a genlight object (these are different data structures), assign the ploidy, and associate the population assignments.

In [ ]:
gl.rubi <- vcfR2genlight(rubi.vcf)
ploidy(gl.rubi) <- 2
pop(gl.rubi) <- pop.data$State
gl.rubi

**12. Index of association using samp.ia**

For SNPs, we don't necessarily want to use all loci for calculating the index of association, because it's overkill. The samp.ia function randomly samples sites to calculate the index of association. The default is to sample 100 SNPs and do this 100 times.

In [ ]:
rubi.ia <- samp.ia(gl.rubi)
rubi.ia

Because that output isn't particularly useful as is, we need to summarize it and better yet compare it to data with known structures. The following code generates data with the same number of individuals and SNPs with varying numbers of linked sites to mimic linkage disequilibrium. The actual and simulated data sets are then compared in a bar plot.

In [ ]:
# Simulated populations
### No structure (admixed pops)
sex <- glSim(94, 608, ploid=2, LD=T)
### Structure (clonal pops)
clone <- glSim(94, 608, n.snp.struc=608, ploid=2, LD = T)
### Semi-clonal - half clonal
semi_clone <- glSim(94, 608, n.snp.struc=304, ploid=2, LD=T)
### Most-clonal - 2/3 clonal
most_clone <- glSim(94, 608, n.snp.struc=405, ploid=2, LD=T)

## IA sex
ia.sex <- samp.ia(sex,quiet = T)
## IA clone
ia.clone <- samp.ia(clone, quiet = T)
## IA.semiclone
ia.semi <- samp.ia(semi_clone, quiet = T)
## IA.mostclone
ia.most <- samp.ia(most_clone, quiet = T)

# Summarizing data frames
d1 <- data.frame(rubi.ia, rep("dataset", length(rubi.ia)))
d2 <- data.frame(ia.sex, rep("sexual", length(ia.sex)))
d3 <- data.frame(ia.clone, rep("clone", length(ia.clone)))
d4 <- data.frame(ia.semi, rep("semi-clone", length(ia.semi)))
d5 <- data.frame(ia.most, rep("most-clone", length(ia.semi)))
colnames(d1) <- c("ia","dset")
colnames(d2) <- c("ia","dset")
colnames(d3) <- c("ia","dset")
colnames(d4) <- c("ia","dset")
colnames(d5) <- c("ia","dset")
ia.total <- rbind(d1, d2, d3, d4, d5)

# Plot
ggplot(ia.total,aes(dset,ia,fill=dset)) + geom_boxplot() + xlab("Dataset") + ylab("Index of association")


**13. Clone correcting SNP datasets.**

For SNP datasets, we need to take a different approach to clone correction. We need to define at what genetic distance to clone correct.

The first step is to move the data into a snpclone object, which can handle clone correction.

In [ ]:
sc.rubi <- as.snpclone(gl.rubi)
sc.rubi

Here is the number of multilocus genotypes in the dataset.

In [ ]:
nmll(sc.rubi)

Now we will caculate the genetic distance between all individuals in the sc.rubi data set and use the cutoff_predictor function to tell us which genetic distances we should consider using to define our multilocus lineages.

In [ ]:
rubi_filtered <- filter_stats(sc.rubi, distance = bitwise.dist, plot = TRUE)
print(farthest_thresh <- cutoff_predictor(rubi_filtered$farthest$THRESHOLDS))
print(average_thresh  <- cutoff_predictor(rubi_filtered$average$THRESHOLDS))
print(nearest_thresh  <- cutoff_predictor(rubi_filtered$nearest$THRESHOLDS))



The resulting histogram of pairwise genetic distances is bimodal. This suggests that there are individuals that are more genetically close than we would expect based on pairwise genetic distances across the dataset (the distribution on the right).

We will use the "farthest_thresh" cutoff as that seems the most likely to capture the potentially clonal isolates in that left node in the histogram.

In [ ]:
mlg.filter(sc.rubi, distance = bitwise.dist, algorithm = "f") <- farthest_thresh
sc.rubi

Now we have 53 multilocus lineages among our 94 isolates.

In [ ]:
nmll(sc.rubi)

**Question 4:** What are your inferences about linkage disequilibrium and sexual reproduction in the *P. rubi* population based on this analysis?

**Question 5:** See assignment for a graph, which shows data for populations of a pathogen on a widely planted crop. The dark diamonds indicate populations in a putative source region and the light diamonds represent populations in other parts of the world. The data shown is: rD, index of association after clone correction plotted against CG:N, which is the number of different clonal groups (CG, i.e., multilocus lineages) scaled by sample size (N). What is figure showing about these populations?